# 실험 02-02 다이오드 전파 정류회로

*******************************************************************************
## 공통 코드

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
from scipy.signal import savgol_filter # 사비츠키-골레이(Savitzky-Golay) 필터

# %matplotlib tk
# %matplotlib inline

def tangent_line(f, x):
    h = 1e-4
    d = (f(x+h) - f(x-h)) / (2*h)
    return lambda t: d*t - d*x + f(x)

def get_simulation_result(file_name, start=0, end=-1):
    with open(file_name, encoding='cp1252') as data_file:
        lines = data_file.read()
        occurrences = lines.count('Step Information:')
        data_file.seek(0)

        line = data_file.readline()
        labels = re.split(', | ,|\t', line)
        labels = [s.strip().upper() for s in labels]

        data = {}

        if (occurrences == 0):
            for label in labels:
                data[label] = []
            
            for line in data_file:
                values = re.split(', | ,|\t', line)
                for i in range(len(values)):
                    value = float(values[i]) * 1000
                    data[labels[i]].append(value)

        else:
            labels_all =[]
            
            lines = data_file.readline() # skip first line starting with 'Step Information:'

            for idx in range(occurrences):
                labels_new = []
                for label in labels:
                    labels_new.append('(%s)@%d' % (label, idx+1))
                labels_all += labels_new

                for label_new in labels_new:
                    data[label_new] = []

                for line in data_file:
                    if (line.startswith('Step Information:')):
                        break
                    values = re.split(', | ,|\t', line)
                    for i in range(len(values)):
                        value = float(values[i]) * 1000
                        data[labels_new[i]].append(value)

            labels = labels_all

        for label in labels:
            data[label] = np.array(data[label][start:end])

    # print("labels = ", end='')
    # print(list(data.keys()))
    for label in list(data.keys()):
        print("data['%s'] : sample number = %d" % (label, len(data[label])))

    return data

def get_oscilloscpoe_result_tektronix(file_name, start=0, end=-1):
    df = pd.read_csv(file_name, header=None, encoding='cp1252',low_memory=False)

    label_ri = df.loc[df.iloc[:,0] == 'Source', 1].index
    ci = 0
    data = {}
    data['TIME'] = df.iloc[start:end, 3].to_numpy().astype(float)
    data['TIME'] -= data['TIME'][0]
    for idx in range(df.shape[1]//6):
        label = df.iloc[label_ri, ci+1].item()
        data[label] = df.iloc[start:end,ci+4].to_numpy().astype(float)
        ci += 6

    # print("labels = ", end='')
    # print(list(data.keys()))
    for label in list(data.keys()):
        print("data['%s'] : sample number = %d" % (label, len(data[label])))

    return data

def print_array(label, values):
    print('%s = [' % label, end='')
    for idx, vd in enumerate(values):
        print('{:7.3f}'.format(vd), end='')
        if (idx+1 != len(values)):
            print(', ', end='')
    print(']')

def print_value(label, value):
    print('%s = ' % label, end='')
    print('{:7.3f}'.format(value))

def draw_plot(xs, ys, label, style_idx, marker_num=16):
    linestyle  = ['-', '-', '-', '-', '-', '-', '-', '-']
    colors     = ['b', 'g', 'r', 'c', 'm', 'y', 'k', 'w']
    markers    = ['o', 'v', 's', 'p', '*', 'h', 'x', 'd']

    markeverys = []
    x_step = (xs.max() - xs.min()) / marker_num
    for idx in range(len(markers)):
        x_start = xs.min() + ((x_step / (len(markers) + 1)) * idx)
        marker_list = []
        for j in range(marker_num):
            tmp_array = np.where(xs > (x_start + (x_step * j)))
            if (len(tmp_array[0]) > 0):
                marker_list.append(tmp_array[0][0])
        markeverys.append(marker_list)

    plt.plot(xs, ys, ls=linestyle[style_idx], color=colors[style_idx], marker=markers[style_idx], markevery=markeverys[style_idx], label=label)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)

def set_plot_size(xrate, yrate):
    if (len(set_plot_size.g_plt_figsize) == 0):
        set_plot_size.g_plt_figsize = plt.rcParams["figure.figsize"]
    size = []
    size.append(set_plot_size.g_plt_figsize[0] * xrate)
    size.append(set_plot_size.g_plt_figsize[1] * yrate)
    plt.figure(figsize=size)

set_plot_size.g_plt_figsize = []

*******************************************************************************
## 실험 1 결과 분석 (실험 과정 4)

### 회로도

* 사용한 소자
    + resistor: 1 kOhm
    + diode: D1N4004

![Markdown-mark](../experiment/circuit_01_01.jpeg)

<!-- ![Markdown-mark](../experiment/m_01_01.png) -->
<!-- ![Markdown-mark](../experiment/m_01_02.png) -->
<!-- ![Markdown-mark](../experiment/m_01_03.png) -->

### 1. 입력 전압 측정 후 실험 결과 파일을 읽는다.

In [ ]:
data = get_oscilloscpoe_result_tektronix('../experiment/m_01_01.csv')

### 2. 실험 결과에서 시간, 입력 전압을 가져온다.

측정치를 가져올 때 1000을 곱해 단위를 밀리로 변환한다.

In [ ]:
times = data['TIME'] * 1000.
vins = data['Sample CH1'] * 1000.

### 3. 시간을 x축, 전압들을 y축으로 하는 그래프를 그린다.

In [ ]:
plt.xlabel('TIME_ms')
plt.ylabel('V_mV')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
draw_plot(times, vins, 'V(IN)', 0)
plt.show()

### 4. 입력 전압의 첨두값을 구한다.

In [ ]:
V_IN_peak_mV = vins.max()

print_value('V_IN_peak_mV', V_IN_peak_mV)

### 5. VA 전압, VB 전압 측정 후 실험 결과 파일을 읽는다.

In [ ]:
data = get_oscilloscpoe_result_tektronix('../experiment/m_01_02.csv')

### 6. 실험 결과에서 시간, VA 전압, VB 전압을 가져온다.

In [ ]:
times = data['TIME'] * 1000
vas = data['Sample CH1'] * 1000
vbs = data['Sample CH2'] * 1000

### 7. 시간을 x축, 전압들을 y축으로 하는 그래프를 그린다.

In [ ]:
plt.xlabel('TIME_ms')
plt.ylabel('V_mV')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
draw_plot(times, vas, 'V(VA)', 1)
draw_plot(times, vbs, 'V(VB)', 2)
plt.show()

### 8. VA 전압의 첨두값, VB 전압의 첨두값을 구한다.

In [ ]:
V_A_peak_mV = vas.max()
V_B_peak_mV = vbs.max()

print_value('V_A_peak_mV', V_A_peak_mV)
print_value('V_B_peak_mV', V_B_peak_mV)

### 9. 출력 전압 측정 후 실험 결과 파일을 읽는다.

In [ ]:
data = get_oscilloscpoe_result_tektronix('../experiment/m_01_03.csv')

### 10. 실험 결과에서 시간, 출력 전압을 가져온다.

측정치를 가져올 때 1000을 곱해 단위를 밀리로 변환한다.

In [ ]:
times = data['TIME'] * 1000
vouts = data['Sample CH2'] * 1000

### 11. 시간을 x축, 전압들을 y축으로 하는 그래프를 그린다.

In [ ]:
plt.xlabel('TIME_ms')
plt.ylabel('V_mV')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
draw_plot(times, vouts, 'V(OUT)', 3)
plt.show()

### 12. 출력 전압의 첨두값, 역방향 첨두값, 리플을 구한다.

In [ ]:
vmax = vouts.max()
vmin = vouts.min()
V_OUT_peak_mV = vmax
V_OUT_r_peak_mV = -vmin
V_OUT_ripple_mV = vmax - vmin

print_value("V_OUT_peak_mV", V_OUT_peak_mV)
print_value("V_OUT_r_peak_mV", V_OUT_r_peak_mV)
print_value("V_OUT_ripple_mV", V_OUT_ripple_mV)

### 13. 시간을 x축, 입력 전압, VA 전압, VB 전압, 출력 전압을 y축으로 하는 그래프를 그린다.

In [ ]:
plt.xlabel('TIME_ms')
plt.ylabel('V_mV')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
draw_plot(times, vins, 'V(IN)', 0)
draw_plot(times, vas, 'V(VA)', 1)
draw_plot(times, vbs, 'V(VB)', 2)
draw_plot(times, vouts, 'V(OUT)', 3)
plt.show()

In [ ]:
plt.xlabel('TIME_ms')
plt.ylabel('V_mV')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
draw_plot(times, vins, 'V(IN)', 0)
draw_plot(times, vas, 'V(VA)', 1)
draw_plot(times, vbs, 'V(VB)', 2)
draw_plot(times, vouts, 'V(OUT)', 3)

ax = plt.gca()
ax.set_xlim([25, 60])
ax.set_ylim([-6000, 6000])
plt.show()

*******************************************************************************
## 실험 2 결과 분석 (실험 과정 5)

### 회로도

* 사용한 소자
    + resistor: 1 kOhm
    + capacitor: 10uF, 22uF, 33uF, 47uF, 56uF
    + diode: D1N4004

![Markdown-mark](../experiment/circuit_02_sweep_c_01.jpeg)

<!-- ![Markdown-mark](../experiment/measurement_02_01_sweep_c.png) -->
<!-- ![Markdown-mark](../experiment/measurement_02_02_sweep_c.png) -->
<!-- ![Markdown-mark](../experiment/measurement_02_03_sweep_c.png) -->
<!-- ![Markdown-mark](../experiment/measurement_02_04_sweep_c.png) -->
<!-- ![Markdown-mark](../experiment/measurement_02_05_sweep_c.png) -->

### 1. 실험 결과 파일을 읽는다.

In [ ]:
data = []
times = []
vouts = []
labels = ['10uF', '22uF', '33uF', '47uF', '56uF']

data.append(get_oscilloscpoe_result_tektronix('../experiment/m_02_01.csv'))
data.append(get_oscilloscpoe_result_tektronix('../experiment/m_02_02.csv'))
data.append(get_oscilloscpoe_result_tektronix('../experiment/m_02_03.csv'))
data.append(get_oscilloscpoe_result_tektronix('../experiment/m_02_04.csv'))
data.append(get_oscilloscpoe_result_tektronix('../experiment/m_02_05.csv'))

### 2. 실험 결과에서 시간, 출력 전압을 가져온다.

In [ ]:
for idx in range(len(labels)):
    times.append(data[idx]['TIME'] * 1000)
    vouts.append(data[idx]['Sample CH1'] * 1000)

### 3. 시간을 x축, 전압들을 y축으로 하는 그래프를 그린다.

In [ ]:
plt.xlabel('TIME_ms')
plt.ylabel('V_OUT_mV')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
for idx in range(len(labels)):
    draw_plot(times[idx], vouts[idx], labels[idx], idx)
plt.show()

In [ ]:
plt.xlabel('TIME_ms')
plt.ylabel('V_OUT_mV')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
for idx in range(len(labels)):
    draw_plot(times[idx], vouts[idx], labels[idx], idx)

ax = plt.gca()
ax.set_xlim([25, 60])
ax.set_ylim([1700, 3500])
plt.show()

In [ ]:
vouts_f = []

plt.xlabel('TIME_ms')
plt.ylabel('V_OUT_mV')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
for idx in range(len(labels)):
    vouts_f.append(savgol_filter(vouts[idx], 200, 2))
    draw_plot(times[idx], vouts_f[idx], labels[idx], idx)

ax = plt.gca()
ax.set_xlim([25, 60])
ax.set_ylim([1700, 3500])
plt.show()

### 4. 출력 전압의 첨두값, 역방향 첨두값, 리플값을 구한다.

In [ ]:
V_OUT_peak_mV = []
V_OUT_r_peak_mV = []
V_OUT_ripple_mV = []
for idx in range(len(labels)):
    idx_start = np.argmax(times[idx] > 25)
    idx_end = np.argmin(times[idx] < 60)

    vmax = vouts_f[idx][idx_start:idx_end].max()
    vmin = vouts_f[idx][idx_start:idx_end].min()

    V_OUT_peak_mV.append(vmax)
    V_OUT_r_peak_mV.append(vmin)
    V_OUT_ripple_mV.append(vmax - vmin)

print("C = ", end='')
print(labels)
print_array("V_OUT_peak_mV", V_OUT_peak_mV)
print_array("V_OUT_r_peak_mV", V_OUT_r_peak_mV)
print_array("V_OUT_ripple_mV", V_OUT_ripple_mV)